In [1]:
import requests
import json
from datetime import timedelta
import numpy as np
import pandas as pd
from datetime import datetime
import os
from IPython.display import clear_output
import local_settings
import hmac
import hashlib

api_key = local_settings.api_key
api_secret = local_settings.api_secret
base_url = 'https://api-testnet.bybit.com'
recv_window = '5000' # increase if alot of traffic

def queryStringUrl(q):
    qstr = ''
    for k,v in q.items():
        qstr += f'{k}={v}&'
    qstr = qstr[0:len(qstr)-1]
    return qstr

def create_header(api_key,api_secret,query):    
    # Step 1: Create param_str with formatted query, timestamp(ms), recvWindow(communication window)
    queryString = queryStringUrl(query)
    now_tstamp = str(int(datetime.now().timestamp())*1000)
    param_str = f'{now_tstamp}{api_key}{recv_window}{queryString}'

    # Step 2: Hash the signature with unique api
    sign = hmac.new(
        bytes(api_secret,'utf-8'),
        bytes(param_str,'utf-8'),
        hashlib.sha256
        ).hexdigest()

    # Step 3: Create header as part of all POST/GET calls
    header = {
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-TIMESTAMP': now_tstamp,
        'X-BAPI-SIGN': sign,
        'X-BAPI-RECV-WINDOW':recv_window}   
    return header

def code_completed():
    os.system("afplay " + 'sample.wav')

## TRADE

Post order AFTER setting tradeMode isolated and leverage

In [2]:
def create_order(perp, side, ordertype, quantity, price, tp=None, sl=None):    
    create_order_url = '/v5/order/create'

    if tp != None and sl != None:
        query = {"category":"linear","symbol":perp,"isLeverage":1,"side":side,"orderType":ordertype,"qty":quantity,"price":price,"takeProfit": tp, "stopLoss": sl}
    elif tp == None and sl == None:
        query = {"category":"linear","symbol":perp,"isLeverage":1,"side":side,"orderType":ordertype,"qty":quantity,"price":price}
    else:
        print("Order can't be created without Stop Loss or Take Profit Level!")

    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.post(url = f'{base_url}{create_order_url}', headers=header, data = query)
    msg = json.loads(msg_recv.content.decode('utf-8')) #turn byte dict string to dict
    print(msg)

---
---
## POSITION

Post Leverage

In [3]:
def post_leverage(perp,lev):
    set_leverage_url = '/v5/position/set-leverage'

    query = {"category":"linear", "symbol":perp, "buyLeverage":str(lev), "sellLeverage":str(lev)}

    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.post(url = f'{base_url}{set_leverage_url}', headers=header, data = query)
    msg = json.loads(msg_recv.content.decode('utf-8')) #turn byte dict string to dict
    print(msg)

---
Post Isloated/Cross Mode

In [4]:
def set_isolated(perp,lev):
    switch_cross_isolated_url = '/v5/position/switch-isolated'

    query = {"category":"linear","symbol":perp,"tradeMode":1, "buyLeverage":str(lev),"sellLeverage":str(lev)}
    # For tradeMode, 0: cross margin. 1: isolated margin

    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.post(url = f'{base_url}{switch_cross_isolated_url}', headers=header, data = query)
    msg = json.loads(msg_recv.content.decode('utf-8')) #turn byte dict string to dict
    print(msg)

----
Get name of all usdt pairs 

In [5]:
def get_usdt_perp_names():
    symbols_url = '/v2/public/symbols'

    query = {"category": "linear"}
    query_string = queryStringUrl(query)

    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.get(url = f'{base_url}{symbols_url}?{query_string}',headers=header)
    msg = json.loads(msg_recv.content.decode('utf-8'))

    pair_names = []
    for token_info in msg['result']:
        pair_names.append(token_info["name"])
    pair_names = [p for p in pair_names if "USDT"== p[len(p)-4:len(p)]]
    return pair_names

---
Get closed pnl of 'symbol' trade

In [6]:
def get_pnl(perp):
    closed_pnl_url = '/v5/position/closed-pnl'
    query = {"category": "linear",
            "symbol":perp}
    query_string = queryStringUrl(query)
    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.get(url = f'{base_url}{closed_pnl_url}?{query_string}',headers=header)
    msg = json.loads(msg_recv.content.decode('utf-8'))
    msg

---
Post TP & SL mode

In [7]:
def set_tpsl_mode(perp):
    set_tpsl_mode_url = '/v5/position/set-tpsl-mode'

    query = {
        "symbol": "BTCUSDT",
        "category": "linear",
        "tpSlMode": "Full" 
    }

    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.post(url = f'{base_url}{set_tpsl_mode_url}', headers=header, data = query)
    msg = json.loads(msg_recv.content.decode('utf-8')) #turn byte dict string to dict
    print(msg)
    # print(msg['retMsg'])

---
Get current positions for 'symbol'

In [8]:
def get_positions(perp):
    positions_url = '/v5/position/list'

    query = {"category": "linear",
            "symbol":perp}
    query_string = queryStringUrl(query)

    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.get(url = f'{base_url}{positions_url}?{query_string}',headers=header)
    msg = json.loads(msg_recv.content.decode('utf-8'))
    print(msg)

---
Get execution/history of 'symbol' trade

In [9]:
def exec_hist(perp):
    execution_url = '/v5/execution/list'
    query = {"category": "linear",
            "symbol":perp}
    query_string = queryStringUrl(query)
    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.get(url = f'{base_url}{execution_url}?{query_string}',headers=header)
    msg = json.loads(msg_recv.content.decode('utf-8'))
    msg

----
----

## ACCOUNT

Get Wallet Balance

In [10]:
def get_balance():
    wall_bal_url = '/v5/account/wallet-balance'
    query = {"accountType": "CONTRACT",
            "coin":'USDT'}
    query_string = queryStringUrl(query)
    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.get(url = f'{base_url}{wall_bal_url}?{query_string}',headers=header)
    msg = json.loads(msg_recv.content.decode('utf-8'))
    return msg

---
Get Coin Greeks


In [11]:
def coin_greeks(coin):
    coin_greeks_url = '/v5/asset/coin-greeks'
    query = {"baseCoin": "ETH"}
    query_string = queryStringUrl(query)
    header = create_header(api_key=api_key,api_secret=api_secret,query=query)
    msg_recv = requests.get(url = f'{base_url}{coin_greeks_url}?{query_string}',headers=header)
    msg = json.loads(msg_recv.content.decode('utf-8'))
    msg

---
---
pybit for getting data:

In [12]:
from pybit import usdt_perpetual
session_unauth = usdt_perpetual.HTTP(endpoint="https://api-testnet.bybit.com")

'''
parameters: token: usdtperp on bybit, 
            interval: 1,5,...,D,W,M , 
            rewind_time: how many days in the past. 
returns pd.DataFrame w/ cols:['symbol','timestamp','date','close']
'''
def create_df(token, interval,rewind_time):
    df = {'symbol':[],'timestamp':[],'date':[],'close':[]}
    curr_ts = int(datetime.now().timestamp())
    start_ts = int((datetime.now() - timedelta(days=rewind_time)).timestamp())

    msg_recv = session_unauth.query_kline(symbol=token, interval = interval, limit = 200, from_time=int(start_ts))

    results = msg_recv['result']
    df = df_filler(df,results)
    last_ts = int(list(df['timestamp'])[-1])

    while curr_ts > last_ts:
        msg_recv = session_unauth.query_kline(symbol=token,interval = interval,limit = 200, from_time=last_ts)
        results = msg_recv['result']
        df = df_filler(df,results)
        last_ts = int(list(df['timestamp'])[-1])

    data = pd.DataFrame(df).drop_duplicates(subset=['date'],keep='first').reset_index(drop=True)
    return data


# Helper function for above functions
def df_filler(dff,res):
    for i in range(len(res)):
        r = res[i]
        dff['symbol'].append(r['symbol'])
        dff['timestamp'].append(r['open_time'])
        dff['date'].append(datetime.strftime(datetime.fromtimestamp(r['open_time']),'%Y-%m-%d %H:%M:%S'))
        dff['close'].append(r['close'])
    return dff